In [9]:
!pip install azureml.core==1.0.41

     |████████████████████████████████| 921kB 2.7MB/s eta 0:00:01
ERROR: azureml-widgets 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is incompatible.
ERROR: azureml-train-core 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is incompatible.
ERROR: azureml-train-automl 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is incompatible.
ERROR: azureml-train-automl 1.0.33 has requirement onnxmltools==1.4.0; python_version < "3.7", but you'll have onnxmltools 1.3.0 which is incompatible.
ERROR: azureml-sdk 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is incompatible.
ERROR: azureml-pipeline-core 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is incompatible.
ERROR: azureml-contrib-notebook 1.0.33 has requirement azureml-core==1.0.33.*, but you'll have azureml-core 1.0.41 which is inc

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%%writefile data_preprocess.py
# Load packages.
import os, sys
from numpy import mean
from math import sqrt
import pandas as pd
df1=pd.read_csv('LengthOfStay.csv')
df1cols=list(df1.columns)
df3=pd.read_csv('MetaData_Facilities.csv')

df13=pd.merge(df1,df3,left_on='facid', right_on='Id')
df13.head()
del df13['Id']
## check classes balance
df13['vdate']=pd.to_datetime(df13['vdate'])
df13['daysofweek_admit']=df13['vdate'].dt.weekday_name
df13['los_numeric']=df13['lengthofstay'].astype('int')
print(df13.dtypes)
## encode string columns
from sklearn.preprocessing import LabelEncoder
le_rcount=LabelEncoder()
df13['rcount']=le_rcount.fit_transform(df13['rcount'])

le_gender=LabelEncoder()
df13['gender']=le_gender.fit_transform(df13['gender'])
le_facility=LabelEncoder()
df13['Name']=le_facility.fit_transform(df13['Name'])
le_daysofweekadmit=LabelEncoder()
df13['daysofweek_admit']=le_daysofweekadmit.fit_transform(df13['daysofweek_admit'])
print(df13.dtypes)
import numpy as np
def turn2cat(item):
    if item>=6 :
        return '>=6_Days'
    else:
        return '<6_Days'
df13['label']=df13['los_numeric'].apply(lambda x: turn2cat(x))

df13.head()

fid=LabelEncoder()
df13['fid']=fid.fit_transform(df13['facid'])
df13['los_numeric']=df13['los_numeric'].astype(float)

df= df13[['rcount', 'gender', 'dialysisrenalendstage', 'asthma',
       'irondef', 'pneum', 'substancedependence', 'psychologicaldisordermajor',
       'depress', 'psychother', 'fibrosisandother', 'malnutrition', 'hemo',
       'hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro',
       'creatinine', 'bmi', 'pulse', 'respiration',
       'secondarydiagnosisnonicd9',  'fid',
       'Capacity', 'Name', 'daysofweek_admit','los_numeric']]
df.to_csv('preprocessed.csv')


Writing data_preprocess.py


In [5]:
%run data_preprocess.py

eid                                    int64
vdate                         datetime64[ns]
rcount                                object
gender                                object
dialysisrenalendstage                  int64
asthma                                 int64
irondef                                int64
pneum                                  int64
substancedependence                    int64
psychologicaldisordermajor             int64
depress                                int64
psychother                             int64
fibrosisandother                       int64
malnutrition                           int64
hemo                                   int64
hematocrit                           float64
neutrophils                          float64
sodium                               float64
glucose                              float64
bloodureanitro                       float64
creatinine                           float64
bmi                                  float64
pulse     

In [2]:
%%writefile split_train_val.py
import pandas as pd
import numpy as np
df13=pd.read_csv('preprocessed.csv')
X= df13[['rcount', 'gender', 'dialysisrenalendstage', 'asthma',
       'irondef', 'pneum', 'substancedependence', 'psychologicaldisordermajor',
       'depress', 'psychother', 'fibrosisandother', 'malnutrition', 'hemo',
       'hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro',
       'creatinine', 'bmi', 'pulse', 'respiration',
       'secondarydiagnosisnonicd9',  'fid',
       'Capacity', 'Name', 'daysofweek_admit']].values
y=df13['los_numeric'].values
#df13.to_csv('merged.csv',index=False)
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

Writing split_train_val.py


In [6]:
%run split_train_val.py

(80000, 27) (20000, 27) (80000,) (20000,)


In [7]:
%%writefile train.py
import timeit
import pickle
import pandas as pd
import numpy as np
x_train=np.load('X_train.npy')
y_train=np.load('y_train.npy')
X_test=np.load('X_test.npy')
y_test=np.load('y_test.npy')
def save_pkl(model, name):
    filename = name
    pickle.dump(model, open(filename, 'wb'))
    
def test_load_pkl(name,X_test):
    loaded_model = pickle.load(open(name, 'rb'))
    result = loaded_model.predict(X_test)
    return result

start = timeit.default_timer()
from sklearn.ensemble import RandomForestRegressor
name='RandomForest_reg'
#run.tag("Description","{} Regressor".format(name))
model = RandomForestRegressor(n_estimators = 57 ,random_state = 0) # n_estimator is the # of trees built
model.fit(X_train, y_train)
filename='LOS_RF_model_nocluster.pkl'.format(name)
save_pkl(model,filename)
y_pred= test_load_pkl(filename,X_test)
mse=mean_squared_error(y_test, y_pred)
print('Mean Squared Error for {} is'.format(name),mse)

Overwriting train.py


In [ ]:
%run train.py

# this should be  retriggered to re-deploy to ACI

In [ ]:
## deploy to ACI 
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')
from azureml.core.model import Model
import sklearn

library_version = "sklearn"+sklearn.__version__.replace(".","x")

model = Model.register(model_path = "LOS_RF_model_nocluster.pkl",
                       model_name = "LOS_RF_model.pkl", # should i give it the same name here for the DevOps retriggering to work?
                       tags = {'area': "LengthOfStay", 'type': "regression", 'version': library_version},
                       description = "RF regression model to predict LengthOfStay",
                       workspace = ws)

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "los", 'type': "regression"},
                                 description = "Image with randomforest regression model")

image = Image.create(name = "los-image0",
                     # this is the model object. note you can pass in 0-n models via this list-type parameter
                     # in case you need to reference multiple models, or none at all, in your scoring script.
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)

In [ ]:

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "los", 'type': "regression"}, 

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'los-aci-service-2nd-0'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
aci_service.get_logs()

In [ ]:
aci_service.scoring_uri

In [ ]:
import numpy as np
import json
##load X_test which we processed from data_preprocess.py --> split_train_val.py from which the model is trained and redeployed 
X_test=np.load("X_test.npy")
y_test=np.load("y_test.npy")
test_sample = json.dumps({'data': [
X_test[0].tolist()
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aci_service.run(input_data=test_sample)
print("actual label {} ,pred label {}".format(str(y_test[0]),str(prediction[0])))